# IRSwaptions in `rvcore`

1. Data Source Configuration
2. Data Fetching
3. Pricing
4. MTM PnL 
5. Backtesting

In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import QuantLib as ql
from datetime import datetime

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

## Data Source Configuration:
- CSV ingest of a timeseries of 'Flatten' Volatility Cubes (columns must follow format `f"{expiry}x{tail} ATMF-{strike offset in bps}"`) with 'Date' column in ISO 8601 extended format with timezone offset (UTC)
- `curve` are CME Clearing Conventions

In [4]:
curve = "USD-SOFR-1D"
swaps_data_source = r"CSV_C:\Users\chris\Project Bond King\rvcore\data\usd_ois.csv"
swaptions_data_source = r"CSV_C:\Users\chris\Project Bond King\rvcore\data\usd_vol_cube.csv"

usd_ois = IRSwaps(
    curve=curve,
    data_source=swaps_data_source,
    ql_interpolation_algo="log_linear",
    pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

usd_swaptions = IRSwaptions(
    data_source=swaptions_data_source,
    irswaps_product=usd_ois,
    error_verbose=True,
    max_n_jobs=-1,
)

BOOTSTRAPPING HISTORICAL IRSWAPS CURVE...: 100%|██████████| 248/248 [00:08<00:00, 29.48it/s]
BOOTSTRAPPING INTRADAY IRSWAPS CURVE...: 0it [00:00, ?it/s]
STRUCTURING FLATTEN CUBE...: 100%|██████████| 337/337 [00:07<00:00, 44.10it/s]


## Data Fetching:

### Quantlib Vol Cube

In [5]:
usd_swaptions.fetch_qlcubes(bdates=[datetime(2025, 5, 9)], to_pydt=True)

BUILDING QL VOL CUBES...: 100%|██████████| 1/1 [00:00<00:00, 9446.63cube/s]


{datetime.datetime(2025, 5, 9, 0, 0): <QuantLib.QuantLib.InterpolatedSwaptionVolatilityCube; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedSwaptionVolatilityCube > *' at 0x0000027F33C575D0> >}

### SCube (`Dict[int, pd.DataFrame]`)
- keyed by ATMF strike offset

In [46]:
dt = datetime(2025, 5, 9)
scube = usd_swaptions.fetch_scubes(bdates=[dt], to_pydt=True)[dt]

scube[0]

Tail,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,30Y
Expiry,,,,,,,,,,,,,
1M,88.738499,108.105399,109.692849,108.740379,107.787908,106.044101,104.295517,102.813896,101.332275,99.850654,97.627615,95.405792,92.389636
3M,99.533164,112.391516,113.661476,110.039966,110.962810,106.425211,107.470418,105.882967,104.295517,102.708066,98.299679,97.945714,94.770812
6M,109.375359,114.296457,114.296457,112.000000,110.962810,107.000000,107.470418,105.935882,104.401347,102.866811,99.313285,98.263204,95.088302
9M,114.613947,116.042653,114.931437,113.000000,110.962810,107.000000,107.629163,106.094628,104.560092,103.025556,100.326890,97.786968,95.247047
1Y,117.788848,117.312613,115.725162,113.000000,110.962810,107.000000,107.629163,106.200458,104.771752,103.343046,100.168145,97.151988,95.564537
2Y,114.455202,113.026496,111.280300,109.000000,108.105399,105.000000,105.406732,104.242602,103.078471,101.914341,98.580694,96.199518,95.088302
3Y,110.804065,109.375359,107.787908,106.000000,104.930497,103.000000,103.025556,102.126001,101.226445,100.326890,96.993243,94.770812,93.659596
4Y,109.000000,106.000000,105.000000,104.000000,103.000000,101.000000,101.438105,100.625404,99.812702,99.000000,95.627353,94.000000,92.000000
5Y,106.676693,104.454262,103.343046,102.000000,101.279360,100.000000,99.850654,99.215674,98.580694,97.945714,94.261462,92.000000,91.278420


### Term Structures

In [47]:
usd_swaptions.vol_smile_plotter(dates=[datetime(2025, 4, 1), datetime(2025, 4, 7), datetime(2025, 5, 9)], tenor="3Mx10Y", use_plotly=True)

BUILDING QL VOL CUBES...: 100%|██████████| 2/2 [00:00<00:00, 5899.16cube/s]


In [48]:
usd_swaptions.term_structure_plotter(dates=[datetime(2025, 4, 1), datetime(2025, 4, 7), datetime(2025, 5, 9)], expiry="1Y")

PLOTTING TERM STRUCTURE...: 100%|██████████| 3/3 [00:04<00:00,  1.38s/it]


In [49]:
usd_swaptions.term_structure_plotter(dates=[datetime(2025, 4, 1), datetime(2025, 4, 7), datetime(2025, 5, 9)], tail="10Y")

PLOTTING TERM STRUCTURE...: 100%|██████████| 3/3 [00:03<00:00,  1.22s/it]


In [50]:
usd_swaptions.vol_surface_plotter(date=datetime(2025, 5, 9), strike_offset=0, use_plotly=True)

In [51]:
usd_swaptions.vol_surface_plotter(date=datetime(2025, 5, 9), tail="10Y", use_plotly=True)

## Timeseries

In [7]:
ts_df = usd_swaptions.timeseries_builder(
    start_date=datetime(2025, 1, 9),
    end_date=datetime(2025, 5, 9),
    queries=[
        IRSwaptionQuery(expiry="3M", tail="10Y", value=IRSwaptionValue.NVOL, structure=IRSwaptionStructure.STRADDLE, structure_kwargs={"strike": "ATMF"}),
        # IRSwaptionQuery(expiry="1Y", tail="10Y", value=IRSwaptionValue.NVOL, structure=IRSwaptionStructure.STRADDLE, structure_kwargs={"strike": "ATMF"}),
        # IRSwaptionQuery(expiry="3M", tail="2Y", value=IRSwaptionValue.NVOL, structure=IRSwaptionStructure.STRADDLE, structure_kwargs={"strike": "ATMF"}),
        # IRSwaptionQuery(expiry="1Y", tail="2Y", value=IRSwaptionValue.NVOL, structure=IRSwaptionStructure.STRADDLE, structure_kwargs={"strike": "ATMF"}),
        # IRSwaptionQuery(expiry="10Y", tail="10Y", value=IRSwaptionValue.NVOL, structure=IRSwaptionStructure.STRADDLE, structure_kwargs={"strike": "ATMF"}),
    ],
    n_jobs=-1,
)

ts_df

PRICING SWAPTIONS…: 0it [00:00, ?it/s]


,USD-SOFR-1D 3Mx10Y ATMF STRADDLE NVOL
Date,
2025-01-09 22:00:00+00:00,179.310143
2025-01-10 22:00:00+00:00,181.235537
2025-01-13 22:00:00+00:00,182.043189
2025-01-14 22:00:00+00:00,182.112887
2025-01-15 22:00:00+00:00,179.873279
...,...
2025-05-05 21:00:00+00:00,177.968590
2025-05-06 21:00:00+00:00,175.776211
2025-05-07 21:00:00+00:00,174.130876


In [56]:
usd_swaptions.timeseries_df_plotter(
    df=ts_df,
    cols_to_plot=[
        "USD-SOFR-1D 3Mx10Y ATMF STRADDLE NVOL",
        "USD-SOFR-1D 3Mx2Y ATMF STRADDLE NVOL",
        "USD-SOFR-1D 1Yx10Y ATMF STRADDLE NVOL",
        "USD-SOFR-1D 1Yx2Y ATMF STRADDLE NVOL",
    ],
    use_plotly=True,
)